1. Pull Missing Assignments through Query - this can be done for the whole district
2. Pull all grades through Query - this can be done for the whole district
3. Calculate # Missing assignments by School and Grade and Student
4. Merge gradebook onto this list
5. Calculate needed metrics for exec dashboard

This file can be used to calculate D's and F's, # missing assignments, # avg missing assignments, which courses have the most missing assignments (top 5)

##### Query to pull missing assignments (whole-district):

LIST STU GBU GBS GBA GBK GBR TCH STU.ID STU.NM STU.GR STU.SC GBK.GN GBR.CD GBK.NM GBK.PD TCH.TE GBU.D1 GBU.CSC GBU.CMK GBA.AN GBA.AD GBA.DD GBA.GC GBS.MK GBA.MX IF GBS.MK = " " AND GBS.MX > 0 AND GBA.GC = 1 AND GBA.DD >= GBR.SD AND GBA.DD <= GBR.ED AND GBK.SC = TCH.SC

Student must have Mark1 = Null for it to be missing.

##### Query to pull all grades (whole-district):
LIST STU GBU GBK GBR STU.SC STU.ID STU.NM STU.GR GBK.PD GBK.NM GBR.CD GBU.CMK GBU.TG GBU.CSC 

Cannot query whole district for all assignments (must only pull missing), query limit is 100,000

##### Query to pull courses (whole-district) make sure to delete duplicates:
LIST CRS E1 C3 CN CO CR MC



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# WHAT MONTH IS THIS DATA?

month = 'October'
date = '10/31/2024'

# CHANGE THE FILE INPUT AND OUTPUT PATHS #
# Input files
grades_file = r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\2024-2025\October\Gradebook.xlsx"
assignments_file = r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\2024-2025\October\Missing Assignments.xlsx"
# Output files
output_missing_assignment_by_student = "C:\\Users\\derek.castleman\Desktop\\MonthlyDataPull\\2024-2025\\October\\Missing Assignment by Student_10312024.xlsx"
output_intermediate = "C:\\Users\\derek.castleman\\Desktop\\MonthlyDataPull\\2024-2025\\October\\Student_Assignments_Grades_10312024.xlsx"
output_final = "C:\\Users\\derek.castleman\\Desktop\\MonthlyDataPull\\2024-2025\\October\\Liping_Grades_File_10312024_final.xlsx"

# Load in the month's previous file for appending
#previous_month = r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\January2023\Student_Assignments_Grades_01312023_blank.xlsx"

#BC and A-G Course reference 
crs = r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\2024-2025\October\Courses.xlsx"


In [ ]:
#Load in the files for missing assignments and student grades
missing = pd.read_excel(assignments_file)

grades = pd.read_excel(grades_file)


In [ ]:
#Sneak peek of missing assignments dataframe
print(f"Length of missing assignments file: {len(missing)}")
missing[:2]

In [ ]:
# Keep only assignments that were due after the start date

#Student ID: 1093132 Health science should end up with 8 and not 12

missing['Start Date'] = pd.to_datetime(missing['Start Date'])
missing['Due Date'] = pd.to_datetime(missing['Due Date'])

missing = missing[missing['Due Date']>= missing['Start Date']]

In [ ]:
a = input('What is the start date you are interested in (mm/dd/yyyy):          ') #Input start date

In [ ]:
a = pd.to_datetime(a) # Change start date to datetime
a

In [ ]:
b = input('What is the end date you are interested in (mm/dd/yyyy):          ') #Input end date

In [ ]:
b = pd.to_datetime(b) # Turn end date to date time
b

In [ ]:
missing = missing[(missing['Due Date'] >=a) & (missing['Due Date'] <=b)]
missing

In [ ]:
#Sneak peek of grades dataframe
print(f"Length of gradebook file: {len(grades)}")
print(set(grades['Term']))
grades[:2]


In [ ]:
#Sum up the number of missing assignments per student per course
bystu = missing.groupby(by=['Student ID',
                            'Student Name',
                            'Name1',
                            'School',
                           'Grade',
                           'Pd'])['Complete'].count().reset_index().rename(columns={'Complete':'# Missing Assignments'})



In [ ]:
bystu

In [ ]:
grades

In [ ]:
# Merge the student grades file with the dataframe containing missing assignments by student and course


m1 = bystu.merge(grades,
                 on=['Student ID',
                     'Name1',
                     'Student Name',
                     'School',
                    'Pd'],
                 how='right').sort_values('Student ID')

# Filter out any courses that are incomplete
m1 = m1[m1['Status'] != 'I']

#Fill in NAN missing assignments with 0
m1['# Missing Assignments'] = m1['# Missing Assignments'].fillna(0)
m1.loc[m1[m1['School'].isin([1,2,4])].index, 'LEA'] = 'Delano'
m1.loc[m1[m1['School'].isin([6,7,8])].index, 'LEA'] = 'Lost Hills'
#Change school name to legible name
schools = {1:"Delano Secondary",
          2:"Delano Secondary",
          4:"Delano Elementary",
          6:"Lost Hills Elementary",
          7:"Lost Hills Secondary",
          8:"Lost Hills Secondary"}

m1.replace({'School':schools},inplace=True)

# Create new column that marks a D or F as 1, anything else as 0
m1['D or F?'] = 0 
m1.loc[m1[m1['OverallMark'].isin(['D','D+','D-','F'])].index, 'D or F?'] = 1

m1['Month'] = month

# Create Grade Categories
m1.loc[m1[m1['OverallMark'].isin(['A','A-','A+'])].index, 'Grade Category'] = 'A'
m1.loc[m1[m1['OverallMark'].isin(['B','B-','B+'])].index, 'Grade Category'] = 'B'
m1.loc[m1[m1['OverallMark'].isin(['C','C-','C+'])].index, 'Grade Category'] = 'C'
m1.loc[m1[m1['OverallMark'].isin(['D','D-','D+','F'])].index, 'Grade Category'] = 'D/F'
m1.loc[m1[m1['OverallMark'].isnull()].index, 'Grade Category'] = 'No Grade'

# Calculate the # of D or Fs per student
df_by_stu = m1.groupby(by='Student ID')['D or F?'].sum().reset_index().rename(columns={'D or F?':'# D/F per student'})

# Merge with original dataframe
m2 = m1.merge(df_by_stu, on='Student ID').reset_index().drop('index',axis=1)


# Create D/F Categories
m2.loc[m2[m2['# D/F per student']==0].index,'D/F Category'] = '0'
m2.loc[m2[m2['# D/F per student']==1].index,'D/F Category'] = '1'
m2.loc[m2[m2['# D/F per student']==2].index,'D/F Category'] = '2'
m2.loc[m2[m2['# D/F per student']>=3].index,'D/F Category'] = '3+'

#Write to file
m2.to_excel(output_missing_assignment_by_student, index=False)

In [ ]:
m2

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Missing Assignments", filename = "Missing Assignments"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(m2)

In [ ]:
set(m2['D/F Category'])

In [ ]:
# Read in Courses file
crs1 = pd.read_excel(crs)

In [ ]:
m2.groupby(by = 'School')['# Missing Assignments'].sum()

In [ ]:
# Add in A-G and BC information from CRS table
m3 = crs1.merge(m2, left_on='Course title', right_on='Name1', how='right')

In [ ]:
# Write dataframe to file for records
m3.to_excel(output_intermediate,index=False)

In [ ]:
m3.groupby(by = 'School')['# Missing Assignments'].sum()

In [ ]:
# Append this dataframe to previous month
#pm = pd.read_excel(previous_month)

pm2 = m3

# Output to file - THIS WILL BE THE STUDENT LEVEL RECORD OF THE CALCULATIONS
pm2.to_excel(output_final, index=False)

In [ ]:
#pm2 = pd.read_excel('~/Documents/WCPA/Executive Dashboard/Output/Student_Assignments_Grades_10282021.xlsx')

In [ ]:
pm2.groupby(by=['School','Month'])['# Missing Assignments'].sum().reset_index().rename(columns={'# Missing Assignments':'# Missing Assignments by School'})

In [ ]:
# Enrollment by school and grade level and all

# Calculate # unique students at each school to get the enrollment numbers

e1 = pm2.groupby(by=['School','Month'])['Student ID'].apply(lambda x: len(x.dropna().unique())).reset_index()
e1.rename(columns={'Student ID':'Enrollment'}, inplace=True)
e1['Grade'] = 99

In [ ]:
e1

In [ ]:
pm2

In [ ]:
# Calculate # unique students at each school and grade level to get the enrollment numbers
e2 = pm2.groupby(by=['School','Month','Grade_y'])['Student ID'].apply(lambda x: len(x.dropna().unique())).reset_index()
e2.rename(columns={'Student ID':'Enrollment'}, inplace=True)

# Calculate # unique students by month to get the enrollment numbers   
e3 = pm2.groupby(by=['Month','LEA'])['Student ID'].apply(lambda x: len(x.dropna().unique())).reset_index()
e3.rename(columns={'Student ID':'Enrollment'}, inplace=True)  
e3['Grade'] = 99
e3['School'] = e3['LEA']

final_e = e1.append(e2, sort=True).append(e3, sort=True).sort_values(['Month','School','Grade'])

In [ ]:
# Number of A,B,C, D/F by school and all

# Calculate # grades in grade categories at each school
g1 = pd.crosstab([pm2['School'],pm2['Month']],pm2['Grade Category'],
           values='Sudent ID', aggfunc=np.count_nonzero).reset_index()
g1['Grade'] = 99


# Calculate # grades in grade categories at each school and grade level
g2 = pd.crosstab([pm2['School'],pm2['Month'],pm2['Grade_y']],pm2['Grade Category'],
           values='Sudent ID', aggfunc=np.count_nonzero).reset_index()


# Calculate # grades in grade categories at each school and grade level and LEA 
g3 = pd.crosstab([pm2['LEA'],pm2['Month']],pm2['Grade Category'],
           values='Sudent ID', aggfunc=np.count_nonzero).reset_index()
g3['Grade'] = 99
g3['School'] = e3['LEA']

final_g = g1.append(g2, sort=True).append(g3, sort=True).sort_values(['Month','School','Grade']).drop('LEA',
                                                                                                      axis=1)


In [ ]:
# % of students with 1, 2 or 3+ failed by school and all
# Calculate # grades in grade categories at each school

d1 = pd.crosstab([pm2['School'],pm2['Month']],pm2['D/F Category'],
           values=pm2['Student ID'],
            aggfunc=lambda x: len(x.dropna().unique()),
            normalize='index').reset_index()
d1['Grade'] = 99

grade_cols = {'0':'D/F Category = 0',
                  '1':'D/F Category = 1',
                  '2':'D/F Category = 2',
                  '3+':'D/F Category = 3+'}

d1.rename(columns=grade_cols, inplace=True)

# Calculate # grades in grade categories at each school and grade level
d2 = pd.crosstab([pm2['School'],pm2['Month'],pm2['Grade_y']],pm2['D/F Category'],
           values=pm2['Student ID'],
            aggfunc=lambda x: len(x.dropna().unique()),
            normalize='index').reset_index()

d2.rename(columns=grade_cols, inplace=True)


# Calculate # grades in grade categories at each school and grade level and LEA 
d3 = pd.crosstab([pm2['LEA'],pm2['Month']],pm2['D/F Category'],
           values=pm2['Student ID'],
            aggfunc=lambda x: len(x.dropna().unique()),
            normalize='index').reset_index()

d3.rename(columns=grade_cols, inplace=True)

d3['Grade'] = 99
d3['School'] = d3['LEA']

final_d = d1.append(d2, sort=True).append(d3, sort=True).sort_values(['Month','School','Grade']).drop('LEA',
                                                                                                      axis=1)



In [ ]:
# Top 5 courses with D's/F's by school - only for most recent
pm3 = pm2[pm2['Month']==month]

df1 = pd.crosstab([pm3['School'],pm3['Name1'],pm3['Month']],pm3['D or F?'],
           values=pm3['Student ID'],
            aggfunc=lambda x: len(x.dropna().unique()),
            normalize='index').reset_index().sort_values(['School',1],ascending=False)
df2 = df1.groupby('School').head(5)


In [ ]:
# Calculate # missing assignments by school by summing all missing assignments across students and course 
# - do this with Power BI instead?

m2 = pm2.groupby(by=['School','Month'])['# Missing Assignments'].sum(). reset_index().rename(columns={'# Missing Assignments':'# Missing Assignments by School'})

m3 = m2.merge(e1, on=['School','Month'])


# Calculate Avg # missing Assignments by school
m3['Avg Missing by School'] = m3['# Missing Assignments by School']/m3['Enrollment']



In [ ]:
# Write all grades and assignment data to excel file
writer = pd.ExcelWriter(output_final)
final_g[['School','Month','Grade','A','B','C','D/F','No Grade']].to_excel(writer,
                                                                          sheet_name='Grades',
                                                                          index=False)
final_e[['School','Month','Grade','Enrollment']].to_excel(writer, sheet_name='Enrollment',index=False)
final_d[['School','Month','Grade','D/F Category = 0','D/F Category = 1','D/F Category = 2','D/F Category = 3+']].to_excel(writer, sheet_name='Ds and Fs',index=False)
m3.to_excel(writer, sheet_name='Missing Assignments',index=False)
df2.to_excel(writer, sheet_name='Top 5 Courses with D or F',index=False)

writer.save()